In [ ]:
!pip install accelerate -U
!pip install pandas
!pip install datasets
!pip install konlpy
!pip install transformers
!pip install torch
!pip install konlpy
!pip install gradio==3.50
!pip install scikit-learn
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab_light_220429.sh
%cd ../


In [ ]:
import pandas as pd
from konlpy.tag import Mecab
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, ElectraForSequenceClassification, BertModel
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from datasets import Dataset
import gradio as gr
from gradio.mix import Parallel
import torch.nn as nn
from kobert_tokenizer import KoBERTTokenizer

# gradio 단수모델

In [ ]:
def predict_bb(text):
    # 카테고리 정의
    categories = ['정치', '경제', '사회', '국제']

    # 텍스트를 Mecab으로 토크나이징합니다.
    mecab = Mecab()
    tokens = mecab.morphs(text)

    # 사전 훈련된 모델 로드
    model_name = "bert-base-multilingual-cased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 토크나이징된 텍스트를 Hugging Face의 토크나이저에 전달합니다.
    inputs = tokenizer(' '.join(tokens), padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

    # 사전 훈련된 모델 로드
    model_multilingual = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4).to(device)

    # 체크포인트 로드
    checkpoint_path = "/home/kannakawai9/sessac_project/epochs_3_t_v_t.pt"
    model_multilingual.load_state_dict(torch.load(checkpoint_path))

    # 모델을 평가 모드로 설정합니다.
    model_multilingual.eval()

    # 예측을 수행합니다.
    with torch.no_grad():
        outputs = model_multilingual(**inputs)

    # 출력에서 가장 높은 값을 가진 클래스를 찾습니다.
    _, predicted = torch.max(outputs.logits, 1)

    # 예측된 클래스를 해당하는 카테고리 이름으로 변환합니다.
    category = categories[predicted.item()]

    return category

# Gradio 인터페이스를 생성합니다.
iface = gr.Interface(
    fn=predict_bb,  # 사용할 함수
    inputs="text",    # 입력 타입
    outputs="text"    # 출력 타입
)

# 인터페이스를 실행합니다.
iface.launch(share=True)

In [ ]:
def predict_kl(text):
    # 사전 훈련된 모델 로드
    model = ElectraForSequenceClassification.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")
    tokenizer = AutoTokenizer.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")

    # classifier의 out_proj 레이어의 출력 차원을 4(레이블 수)로 조정
    model.classifier.out_proj = nn.Linear(in_features=model.classifier.dense.out_features, out_features=4)

    # 모델의 num_labels 속성 업데이트
    model.num_labels = 4

    # 체크포인트 로드
    checkpoint_path = "/home/kannakawai9/best_not_one_koelectra12.pt"
    model.load_state_dict(torch.load(checkpoint_path))

    # 레이블 딕셔너리 정의
    label_dict = {'정치': 0, '사회': 1, '경제': 2, '국제': 3}

    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    categories = ['정치', '사회', '경제', '국제']
    mecab = Mecab()
    
    # 텍스트를 Mecab으로 토크나이징합니다.
    tokens = mecab.morphs(text)

    # 토크나이징된 텍스트를 Hugging Face의 토크나이저에 전달합니다.
    inputs = tokenizer(' '.join(tokens), padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

    # 모델을 평가 모드로 설정합니다.
    model.eval()

    # 예측을 수행합니다.
    with torch.no_grad():
        outputs = model(**inputs)

    # 출력에서 가장 높은 값을 가진 클래스를 찾습니다.
    _, predicted = torch.max(outputs.logits, 1)

    # 예측된 클래스를 해당하는 카테고리 이름으로 변환합니다.
    category = categories[predicted.item()]

    return category

# Gradio 인터페이스를 생성합니다.
iface = gr.Interface(
    fn=predict_kl,  # 사용할 함수
    inputs="text",    # 입력 타입
    outputs="text"    # 출력 타입
)

# 인터페이스를 실행합니다.
iface.launch(share=True)



In [ ]:
def predict_kb(text):
    # 사전 훈련된 모델 로드
    model = BertModel.from_pretrained('skt/kobert-base-v1')

    # CustomModel 정의
    class CustomModel(nn.Module):
        def __init__(self, model, num_classes):
            super(CustomModel, self).__init__() # 부모 클래스 초기화
            self.bert = model
            self.dropout = nn.Dropout(p=0.5)
            self.fc = nn.Linear(768, num_classes)

        def forward(self, input_ids, attention_mask, token_type_ids):
            output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            last_hidden_state = output['last_hidden_state']
            output = self.dropout(last_hidden_state[:, 0, :])
            output = self.fc(output)
            return output

    # CustomModel 인스턴스 생성
    model = CustomModel(model, num_classes=4)

    # 체크포인트 로드
    checkpoint_path = "/home/kannakawai9/sessac_project/kobert.pt"
    model.load_state_dict(torch.load(checkpoint_path))

    # 사전 훈련된 모델 로드
    tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

    # 레이블 딕셔너리 정의
    label_dict = {0: '정치', 1: '사회', 2: '경제', 3: '국제'}

    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    categories = ['정치', '사회', '경제', '국제']
    mecab = Mecab()
    
    # 텍스트를 Mecab으로 토크나이징합니다.
    tokens = mecab.morphs(text)

    # 토크나이징된 텍스트를 Hugging Face의 토크나이저에 전달합니다.
    inputs = tokenizer.encode_plus(
        ' '.join(tokens),
        return_tensors='pt',     # 텐서로 반환
        truncation=True,         # 입력 시퀀스가 최대 길이를 초과할 때 자를지 여부 결정
        padding='max_length',
        max_length=512, # 최대 입력 길이
        add_special_tokens=True
    )
    inputs = {k:v.to(device) for k,v in inputs.items()}

    # 모델을 평가 모드로 설정합니다.
    model.eval()

    # 예측을 수행합니다.
    with torch.no_grad():
        outputs = model(**inputs)

    # 출력에서 가장 높은 값을 가진 클래스를 찾습니다.
    _, predicted = torch.max(outputs, 1)

    # 예측된 클래스를 해당하는 카테고리 이름으로 변환합니다.
    category = categories[predicted.item()]

    return category

# Gradio 인터페이스를 생성합니다.
iface = gr.Interface(
    fn=predict_kb,  # 사용할 함수
    inputs="text",    # 입력 타입
    outputs="text"    # 출력 타입
)

# 인터페이스를 실행합니다.
iface.launch(share=True)

In [ ]:
def predict_kr(text):
    # 사전 훈련된 모델 로드
    model = ElectraForSequenceClassification.from_pretrained("klue/roberta-base")
    tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

    # classifier의 out_proj 레이어의 출력 차원을 4(레이블 수)로 조정
    model.classifier.out_proj = nn.Linear(in_features=model.classifier.dense.out_features, out_features=4)

    # 모델의 num_labels 속성 업데이트
    model.num_labels = 4

    # 체크포인트 로드
    checkpoint_path = "/home/kannakawai9/klue_roberta.pt"
    model.load_state_dict(torch.load(checkpoint_path))

    # 레이블 딕셔너리 정의
    label_dict = {0: '정치', 1: '사회', 2: '경제', 3: '국제'}

    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    categories = ['정치', '사회', '경제', '국제']
    mecab = Mecab()
    
    # 텍스트를 Mecab으로 토크나이징합니다.
    tokens = mecab.morphs(text)

    # 토크나이징된 텍스트를 Hugging Face의 토크나이저에 전달합니다.
    inputs = tokenizer(' '.join(tokens), padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

    # 모델을 평가 모드로 설정합니다.
    model.eval()

    # 예측을 수행합니다.
    with torch.no_grad():
        outputs = model(**inputs)

    # 출력에서 가장 높은 값을 가진 클래스를 찾습니다.
    _, predicted = torch.max(outputs.logits, 1)

    # 예측된 클래스를 해당하는 카테고리 이름으로 변환합니다.
    category = categories[predicted.item()]

    return category

# Gradio 인터페이스를 생성합니다.
iface = gr.Interface(
    fn=predict_kr,  # 사용할 함수
    inputs="text",    # 입력 타입
    outputs="text"    # 출력 타입
)

# 인터페이스를 실행합니다.
iface.launch(share=True)

# gradio 복수 모델

In [ ]:
iface_bb = gr.Interface(fn=predict_bb, inputs="textbox", outputs=[gr.Textbox(label="bert-base-multilingual-cased", lines=3)])
iface_kl = gr.Interface(fn=predict_kl, inputs="textbox", outputs=[gr.Textbox(label="copycats/koelectra-base-v3-generalized-sentiment-analysis", lines=3)])
iface_kb = gr.Interface(fn=predict_kb, inputs="textbox", outputs=[gr.Textbox(label="skt/kobert-base-v1", lines=3)])
iface_kr = gr.Interface(fn=predict_kr, inputs="textbox", outputs=[gr.Textbox(label="klue/roberta-base", lines=3)])

# Parallel 인터페이스를 정의합니다.
iface = Parallel(iface_bb, iface_kl, iface_kb, iface_kr)

# Gradio 인터페이스를 실행합니다.
iface.launch(share=True)
